In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv('bank.csv')
df

In [ ]:
df=pd.read_csv('bank.csv', sep =';')
df

In [ ]:
df.isnull().sum()

In [ ]:
bal = df['balance']
bal

In [ ]:
def balanceator (x):
    if x < 72:
        return 'Class E'
    elif x >= 72 and x <448:
        return 'Class D'
    elif x >= 448 and x < 1428:
        return 'Class C'
    elif x >= 1428 and x < bal.quantile(0.99):
        return 'Class B'
    else:
        return 'Class A'
    


In [ ]:
def wrangle () :
    df = pd.read_csv('bank.csv', sep=';') 
    df['y'] = df['y'].apply(lambda x: True if x == 'yes' else False) 
    df['default'] = df['default'].apply(lambda x: True if x == 'yes' else False) 
    df['balance_class'] = df['balance'].apply(lambda x: balanceator(x)) 
    df['housing'] = df['housing'].apply(lambda x: True if x == 'yes' else False)
    df['loan'] = df['loan'].apply(lambda x: True if x == 'yes' else False) 
    df['previous_bool'] = df['previous'].apply(lambda x: True if x != 0 else False) 
    
    
    to_drop =['previous', 'day', 'poutcome', 'pdays'] 
    df.drop(columns= to_drop, inplace=True)
    
    
    return df

In [ ]:
new_df = wrangle()
new_df

In [ ]:
x=new_df.drop (columns=['y', 'balance', 'duration'])
y=new_df['y']
print (x)
print (y)


In [ ]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()
x=oe.fit_transform(x)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split (x,y,test_size=0.2, random_state=42)


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
gsc_dst= GridSearchCV(DecisionTreeClassifier(random_state=42), {}, n_jobs=-1, cv=10, refit="recall")

gsc_dst.fit(x_train, y_train)


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_estimator(gsc_dst, x_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred=gsc_dst.predict(x_test)
print ('The Confusion matrix is', confusion_matrix(y_test,y_pred))
print ('The Accuracy score is', accuracy_score(y_test, y_pred))

In [ ]:
params_gsc_dst = {
    "max_depth": [5, 10, 15, 20, 25, 30, None],
    "criterion": ["gini","entropy"], 
    "min_samples_split": [2,3], 
    "min_samples_leaf": [1,2]
}

In [ ]:
newmodel_gsc_dst = GridSearchCV(
    DecisionTreeClassifier(random_state=42), 
    params_gsc_dst,
    cv=10, 
    verbose=2
)

In [ ]:
newmodel_gsc_dst.fit(x_train, y_train)

In [ ]:
ConfusionMatrixDisplay.from_estimator(newmodel_gsc_dst, x_test, y_test)

In [ ]:
new_ypred=newmodel_gsc_dst.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report
print ('The classification report is', classification_report(y_test,new_ypred))
print ('The new accuracy score is', accuracy_score(y_test, new_ypred))
print ('The new confusion matrix is', confusion_matrix(y_test, new_ypred))